In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from fractions import Fraction
import numpy as np
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\Quantomatic.jar'
zx.tikz.tikzit_location = r'C:\Users\John\Documents\tikzit\tikzit.exe'

In [66]:
import json
import ipywidgets as widgets
from traitlets import Unicode, validate, Bool, Int, Float
from IPython.display import display, HTML

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)


@widgets.register
class Email(widgets.DOMWidget):
    _view_name = Unicode('EmailView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

    # Attributes
    value = Unicode('example@example.com', help="The email value.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes.").tag(sync=True)

@widgets.register
class ZXGraphWidget(widgets.DOMWidget):
    _view_name = Unicode('ZXGraphView').tag(sync=True)
    _model_name = Unicode('ZXGraphModel').tag(sync=True)
    _view_module = Unicode('zxgraph').tag(sync=True)
    _model_module = Unicode('zxgraph').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _model_module_version = Unicode('0.1.0').tag(sync=True)
    
    graph_json = Unicode('{"nodes": [], "links": []}').tag(sync=True)
    graph_id = Unicode('0').tag(sync=True)
    graph_width = Float(600.0).tag(sync=True)
    graph_height = Float(400.0).tag(sync=True)
    graph_node_size = Float(5.0).tag(sync=True)
    
    def __init__(self, graph, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.observe(self.handle_graph_change, 'graph_json')
        self.graph = graph
        self.changes = []
    
    def handle_graph_change(self, change):
        js = json.loads(change['new'])
        scale = self.graph.scale
        marked = self.graph.vertex_set()
        for n in js["nodes"]:
            v = int(n["name"])
            r = float(n["x"])/scale -1
            q = float(n["y"])/scale -2
            t = int(n["t"])
            phase = s_to_phase(n["phase"])
            if v not in marked:
                self.graph.add_vertex(t, q, r, phase)
            else: 
                marked.remove(v)
                self.graph.set_position(v, q, r)
                self.graph.set_phase(v, phase)
                self.graph.set_type(v, t)
        self.graph.remove_vertices(marked)
        marked = self.edge_set()
        for e in js["links"]:
            s = int(e["source"])
            t = int(e["target"])
            et = int(e["t"])
            if self.graph.connected(s,t):
                f = self.graph.edge(s,t)
                marked.remove(f)
                self.graph.set_edge_type(f, et)
            else:
                self.graph.add_edge((s,t),et)
        self.graph.remove_edges(marked)

In [67]:
c = zx.Circuit(3)
c.add_gate("CNOT",0,1)
c.add_gate("HAD",2)
g = c.to_graph()

In [59]:
def phase_to_s(a):
    if not a: return ''
    if not isinstance(a, Fraction):
        a = Fraction(a)
    ns = '' if a.numerator == 1 else str(a.numerator)
    ds = '' if a.denominator == 1 else '/' + str(a.denominator)

    # unicode 0x03c0 = pi
    return ns + '\u03c0' + ds

def s_to_phase(s):
    if not s: return Fraction(0)
    s = s.replace('\u03c0', '')
    if s.find('/') != -1:
        a,b = s.split("/", 2)
        if not a: return Fraction(1,int(b))
        return Fraction(int(a),int(b))
    if not s: return Fraction(1)
    return Fraction(int(s))

def graph_to_json(g, scale):
    nodes = [{'name': str(v),
              'x': (g.row(v) + 1) * scale,
              'y': (g.qubit(v) + 2) * scale,
              't': g.type(v),
              'phase': phase_to_s(g.phase(v)) }
             for v in g.vertices()]
    links = [{'source': str(g.edge_s(e)),
              'target': str(g.edge_t(e)),
              't': g.edge_type(e) } for e in g.edges()]
    return json.dumps({'nodes': nodes, 'links': links})

_d3_display_seq = 0
def make_widget(g, scale=None):
    global _d3_display_seq
    _d3_display_seq += 1
    seq = _d3_display_seq

    if scale == None:
        scale = 800 / (g.depth() + 2)
        if scale > 50: scale = 50
        if scale < 20: scale = 20
    
    g.scale = scale
    
    node_size = 0.2 * scale
    if node_size < 2: node_size = 2

    w = (g.depth() + 2) * scale
    h = (g.qubit_count() + 3) * scale
    
    js = graph_to_json(g, scale)
    w = ZXGraphWidget(g, graph_json = js, graph_id = str(seq), 
                      graph_width=w, graph_height=h, graph_node_size=node_size)
    return w

In [73]:
w = make_widget(g)
w

ZXGraphWidget(graph_height=308.0, graph_id='5', graph_json='{"nodes": [{"name": "0", "x": 50.0, "y": 100.0, "t…

In [63]:
zx.d3.draw(w.graph)

In [ ]:
%%javascript
require.undef('mytestfunction')

define('mytestfunction', [], function() {
    console.log("Testing my function");
    return {hoi: function() {console.log("blabla")}};
});


require(['mytestfunction'], function(test) {
    console.log("Testing")
    test.hoi()
    return {}
});

In [74]:
%%javascript
require.undef('zxgraph');

define('zxgraph', ["@jupyter-widgets/base", "makegraph"], function(widgets,makegraph) {
    
    var ZXGraphModel = widgets.DOMWidgetModel.extend({
        defaults: _.extend(widgets.DOMWidgetModel.prototype.defaults(), {
            _model_name: 'ZXGraphModel',
            _view_name: 'ZXGraphView',
            _model_module: 'zxgraph',
            _view_module: 'zxgraph',
            _model_module_version: '0.1.0',
            _view_module_version: '0.1.0',
            graph_json: '{"nodes": [], "links": []}',
            graph_id: '0',
            graph_width: 600.0,
            graph_height: 400.0,
            graph_node_size: 5.0
        })
    });
    
    var ZXGraphView = widgets.DOMWidgetView.extend({
        
        render: function() {
            var btn = document.createElement('button')
            btn.textContent = 'click me'
            btn.onclick = this.value_changed.bind(this);
            this.el.appendChild(btn)
            var mydiv = document.createElement('div');
            mydiv.setAttribute('style', 'overflow:auto');
            var div_id = 'graph-output-' + this.model.get('graph_id');
            mydiv.setAttribute('id', div_id);
            //div.textContent = 'blabla';
            this.el.appendChild(mydiv);
            this.graph = JSON.parse(this.model.get('graph_json'));
            var w = this.model.get("graph_width");
            var h = this.model.get("graph_height");
            var node_size = this.model.get("graph_node_size");
            makegraph.showGraph(mydiv, this.graph, w, h, node_size, false, false);
            //this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            console.log("Clicked")
            //console.log(this.graph)
            this.model.set('graph_json', JSON.stringify(this.strip_graph()));
            this.model.save_changes();
        },
        strip_graph: function() {
            var g = {links: [], nodes: []}
            this.graph.nodes.forEach(function(d) {
                g.nodes.push({"name": d.name, "x":d.x, "y": d.y, "t": d.t, "phase": d.phase})
            });
            this.graph.links.forEach(function(d) {
               g.links.push({"source": d.source.name, "target": d.target.name, "t":d.t}) 
            });
            return g
        }
    });
    
    return {
        ZXGraphModel: ZXGraphModel,
        ZXGraphView: ZXGraphView
    };
});

<IPython.core.display.Javascript object>

In [ ]:
%%javascript
require.undef('hello');

define('hello', ["@jupyter-widgets/base"], function(widgets) {
    
    var HelloView = widgets.DOMWidgetView.extend({
        
        render: function() { 
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            this.el.textContent = this.model.get('value'); 
        },
    });
    
    return {
        HelloView : HelloView
    };
});

In [ ]:
%%javascript
require.undef('email_widget');

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {

    var EmailView = widgets.DOMWidgetView.extend({

        // Render the view.
        render: function() {
            this.email_input = document.createElement('input');
            this.email_input.type = 'email';
            this.email_input.value = this.model.get('value');
            this.email_input.disabled = this.model.get('disabled');

            this.el.appendChild(this.email_input);

            // Python -> JavaScript update
            this.model.on('change:value', this.value_changed, this);
            this.model.on('change:disabled', this.disabled_changed, this);

            // JavaScript -> Python update
            this.email_input.onchange = this.input_changed.bind(this);
        },

        value_changed: function() {
            this.email_input.value = this.model.get('value');
        },

        disabled_changed: function() {
            this.email_input.disabled = this.model.get('disabled');
        },

        input_changed: function() {
            this.model.set('value', this.email_input.value);
            this.model.save_changes();
        },
    });

    return {
        EmailView: EmailView
    };
});

In [ ]:
email = Email(value='john.doe@domain.com', disabled=False)
email

In [72]:
%%javascript

require.config({baseUrl: "../js", paths: {d3: "d3.v4.min"} });

require.undef('makegraph')

define('makegraph', ['d3'], function(d3) {
        
    // styling functions
    function nodeColor(t) {
        if (t == 0) return "black";
        else if (t == 1) return "green";
        else if (t == 2) return "red";
        else if (t == 3) return "yellow";
    }

    function edgeColor(t) {
        if (t == 1) return "black";
        else if (t == 2) return "#08f";
    }

    function nodeStyle(selected) {
        return selected ? "stroke-width: 2px; stroke: #00f" : "stroke-width: 1.5px";
    }

    return {
    showGraph: function(tag, graph, width, height, node_size, auto_hbox, show_labels) {
        var ntab = {};

        graph.nodes.forEach(function(d) {
            ntab[d.name] = d;
            d.selected = false;
            d.previouslySelected = false;
            d.nhd = [];
        });

        var spiders_and_boundaries = graph.nodes.filter(function(d) {
            return d.t != 3;
        });

        graph.links.forEach(function(d) {
            var s = ntab[d.source];
            var t = ntab[d.target];
            d.source = s;
            d.target = t;
            s.nhd.push(t);
            t.nhd.push(s);
        });

        var shiftKey;

        // SETUP SVG ITEMS

        var svg = d3.select(tag)
            .attr("tabindex", 1)
            .on("keydown.brush", function() {shiftKey = d3.event.shiftKey || d3.event.metaKey;})
            .on("keyup.brush", function() {shiftKey = d3.event.shiftKey || d3.event.metaKey;})
            .each(function() { this.focus(); })
            .append("svg")
            .attr("style", "max-width: none; max-height: none")
            .attr("width", width)
            .attr("height", height);

        var link = svg.append("g")
            .attr("class", "link")
            .selectAll("line")
            .data(graph.links)
            .enter().append("line")
            .attr("stroke", function(d) { return edgeColor(d.t); })
            .attr("style", "stroke-width: 1.5px");

        var brush = svg.append("g")
            .attr("class", "brush");

        var node = svg.append("g")
            .attr("class", "node")
            .selectAll("g")
            .data(graph.nodes)
            .enter().append("g")
            .attr("transform", function(d) {
                return "translate(" + d.x + "," + d.y +")";
            });

        node.filter(function(d) { return d.t != 3; })
            .append("circle")
            .attr("r", function(d) {
               if (d.t == 0) return 0.5 * node_size;
               else return node_size;
            })
            .attr("fill", function(d) { return nodeColor(d.t); })
            .attr("stroke", "black");

        var hbox = node.filter(function(d) { return d.t == 3; });

        hbox.append("rect")
            .attr("x", -0.75 * node_size).attr("y", -0.75 * node_size)
            .attr("width", node_size * 1.5).attr("height", node_size * 1.5)
            .attr("fill", function(d) { return nodeColor(d.t); })
            .attr("stroke", "black");

        node.filter(function(d) { return d.phase != ''; })
            .append("text")
            .attr("y", 0.7 * node_size + 14)
            .text(function (d) { return d.phase })
            .attr("text-anchor", "middle")
            .attr("font-size", "12px")
            .attr("font-family", "monospace")
            .attr("fill", "#00d");

        if (show_labels) {
            node.append("text")
                .attr("y", -0.7 * node_size - 5)
                .text(function (d) { return d.name; })
                .attr("text-anchor", "middle")
                .attr("font-size", "8px")
                .attr("font-family", "monospace")
                .attr("fill", "#ccc");
        }

        function update_hboxes() {
            if (auto_hbox) {
                var pos = {};
                hbox.attr("transform", function(d) {
                    // calculate barycenter of non-hbox neighbours, then nudge a bit
                    // to the NE.
                    var x=0,y=0,sz=0;
                    for (var i = 0; i < d.nhd.length; ++i) {
                        if (d.nhd[i].t != 3) {
                            sz++;
                            x += d.nhd[i].x;
                            y += d.nhd[i].y;
                        }
                    }

                    if (sz != 0) {
                        x = (x/sz) + 20;
                        y = (y/sz) - 20;

                        while (pos[[x,y]]) {
                            x += 20;
                        }
                        d.x = x;
                        d.y = y;
                        pos[[x,y]] = true;
                    }

                    return "translate("+d.x+","+d.y+")";
                });
            }
        }

        update_hboxes();

        link.attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });

        // EVENTS FOR DRAGGING AND SELECTION
        
        svg.on("mousedown", function(d) {console.log("mousedown on svg")});
        svg.on("mouseup", function(d) {console.log("clicked svg")});

        node.on("mousedown", function(d) {
                if (shiftKey) {
                    d3.select(this).select(":first-child").attr("style", nodeStyle(d.selected = !d.selected));
                    d3.event.stopImmediatePropagation();
                } else if (!d.selected) {
                    node.select(":first-child").attr("style", function(p) { return nodeStyle(p.selected = d === p); });
                }
            })
            .call(d3.drag().on("drag", function(d) {
                var dx = d3.event.dx;
                var dy = d3.event.dy;
                // node.filter(function(d) { return d.selected; })
                //     .attr("cx", function(d) { return d.x += dx; })
                //     .attr("cy", function(d) { return d.y += dy; });
                node.filter(function(d) { return d.selected; })
                    .attr("transform", function(d) {
                        d.x += dx;
                        d.y += dy;
                        return "translate(" + d.x + "," + d.y +")";
                    });

                update_hboxes();

                link.filter(function(d) { return d.source.selected ||
                                            (auto_hbox && d.source.t == 3); })
                    .attr("x1", function(d) { return d.source.x; })
                    .attr("y1", function(d) { return d.source.y; });

                link.filter(function(d) { return d.target.selected ||
                                            (auto_hbox && d.target.t == 3); })
                    .attr("x2", function(d) { return d.target.x; })
                    .attr("y2", function(d) { return d.target.y; });

                // text.filter(function(d) { return d.selected; })
                //     .attr("x", function(d) { return d.x; })
                //     .attr("y", function(d) { return d.y + 0.7 * node_size + 14; });
            }));

        brush.call(d3.brush()
            .extent([[0, 0], [width, height]])
            .on("start", function() {
                if (d3.event.sourceEvent.type !== "end") {
                    node.select(":first-child").attr("style", function(d) {
                        return nodeStyle(
                            d.selected = d.previouslySelected = shiftKey &&
                            d.selected);
                    });
                }
            })
            .on("brush", function() {
                if (d3.event.sourceEvent.type !== "end") {
                    var selection = d3.event.selection;
                    node.select(":first-child").attr("style", function(d) {
                        return nodeStyle(d.selected = d.previouslySelected ^
                            (selection != null
                            && selection[0][0] <= d.x && d.x < selection[1][0]
                            && selection[0][1] <= d.y && d.y < selection[1][1]));
                    });
                }
            })
            .on("end", function() {
                if (d3.event.selection != null) {
                    d3.select(this).call(d3.event.target.move, null);
                }
            }));
    }};
});


<IPython.core.display.Javascript object>